In [1]:
import requests
from bs4 import BeautifulSoup as soup
import re
import lxml

## Getting links and metadata from https://www.n-tv.de/news.xml

In [69]:
def extract_meta_data():
    news = soup(requests.get('https://www.n-tv.de/news.xml').text, features="xml")
    
    keywords = ['Ukraine','Russland', 'Wladimir Putin']  # ToDo Keywords spezifizieren
    
    filtered_news = [url for url in news.find('urlset').find_all('url', recursive=False) if len(set(keywords).intersection(set(url.find('news:keywords').text.split(', '))))!=0]
    
    return_list = []
    for new in filtered_news:
        dictionary = {
        'url' : new.find('loc').text,
        'title' : new.find('news:title').text,
        'date' : new.find('news:publication_date').text,
        'keywords' : new.find('news:keywords').text
        }
        dictionary = { **dictionary, 'figcaption' : new.find('image:caption').text } if new.find('image:caption') else dictionary
        return_list += [dictionary]
        
    return return_list


## Getting text and ... from urls

In [67]:
url = extract_meta_data()[0]['url']
url

'https://www.n-tv.de/politik/Moskau-nimmt-Sarmat-Rakete-ins-Arsenal-auf-article23285525.html'

In [66]:
article_soup = soup(requests.get(url).text)

In [70]:
article_soup

<!DOCTYPE html>
<html class="no-js" lang="de">
<head>
<title>Ab Herbst im Dienst: Moskau nimmt Sarmat-Rakete ins Arsenal auf - n-tv.de</title>
<base href="https://www.n-tv.de/politik/Moskau-nimmt-Sarmat-Rakete-ins-Arsenal-auf-article23285525.html"/>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="n-tv NACHRICHTEN" name="author"/>
<meta content="n-tv Nachrichten" name="copyright"/>
<meta content="Vor wenigen Tagen meldet Russland den ersten erfolgreichen Test seiner Interkontinentalrakete Sarmat. Mit einer Reichweite von mehreren Tausend Kilometern kann die Rakete jedes Ziel auf der Erde angreifen. Spätestens im Herbst soll das Waffensystem den Streitkräften zur Verfügung stehen." name="description"/>
<meta content="Politik, Russland, Raketentests, Atomwaffen, Wladimir Putin" name="keywords"/>
<meta content="Politik, Russland, Raketentests, Atomwaffen, Wladimir Putin" name